In [2]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import seaborn as sns

# Set the path to the CSV file
file_path = r'C:\Users\r_mat\OneDrive\Documents\GitHub\Project_1\Data\Crime_Data_from_2020_to_Present.csv'

# Read the CSV file into a DataFrame
crime_data = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to understand the structure of the data
crime_data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


# 4. Demographics Analysis

### Visualization 11: 
    - Count plot or pie chart showing the distribution of crime victims by gender: This visualization will display the proportion or count of crime victims based on gender. A pie chart can illustrate the relative percentage of male, female, or other gender victims. A count plot can showcase the absolute count of victims by gender.
### Visualization 12: 
    - Scatter plot with a linear regression line to explore the correlation between the age of the victim and the count of a specific crime type: This plot aims to analyze the relationship between victim age and the occurrence of a particular crime type. It'll help determine if there's any correlation between age and the frequency of a specific crime.
### Visualization 13: 
    - Histogram or kernel density plot showing the distribution of victim ages: This visualization will depict the distribution of victim ages. A histogram will display age groups and their frequency, while a kernel density plot will show the probability density function of victim ages, offering insights into the age distribution of crime victims.
### Visualization 14: 
    - Stacked bar plot or heat map showing the correlation between crime types and victim demographics: This plot will explore the relationship between different crime types and victim demographics (e.g., age groups, gender). A stacked bar plot can display the breakdown of crime types within each demographic category, while a heat map can indicate the strength of correlation between crime types and victim demographics using colors or shades.

In [ ]:
# Visualization 11:
# Count plot chart showing the distribution of crime victims by gender

crime_data['Vict Sex'].value_counts().plot(kind='bar')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Distribution of Crime Victims by Gender')
plt.show()

In [ ]:
# For a Pie Chart

plt.figure(figsize=(8, 8))
crime_data['Vict Sex'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Crime Victims by Gender')
plt.ylabel('')  # Remove the label on the y-axis

plt.show()

In [3]:
# Visualization 12: 
# Scatter plot with a linear regression line to explore the correlation between the 
# age of the victim and the count of a specific crime type.

# Overview of crime counts by age
crime_counts_by_age = crime_data['Vict Age'].value_counts()

# Display the top 10 areas with the highest crime counts
top_crime_age = crime_counts_by_age.head(10)
print("Top 10 Age with the Highest Crime Counts:")
print(top_crime_age)

# Overview of crime counts by specific crime type
crime_counts_by_type = crime_data['Crm Cd Desc'].value_counts()

# Display the top 10 specific crime types with the highest counts
top_crime_types = crime_counts_by_type.head(15)
print("\nTop 10 Crime Types with the Highest Counts:")
print(top_crime_types)

Top 10 Age with the Highest Crime Counts:
Vict Age
0     211842
30     19421
35     19008
31     18603
29     18552
28     18266
32     18066
33     17557
27     17534
26     16859
Name: count, dtype: int64

Top 10 Crime Types with the Highest Counts:
Crm Cd Desc
VEHICLE - STOLEN                                            91473
BATTERY - SIMPLE ASSAULT                                    67976
THEFT OF IDENTITY                                           53467
BURGLARY FROM VEHICLE                                       52611
BURGLARY                                                    51961
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)     51826
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT              48876
THEFT PLAIN - PETTY ($950 & UNDER)                          43402
INTIMATE PARTNER - SIMPLE ASSAULT                           42729
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)             32875
THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)         30716
ROBBERY   

In [4]:
crime_counts_by_age = crime_data['Vict Age'].value_counts()
crime_counts_by_type = crime_data['Crm Cd Desc'].value_counts()

# Create a DataFrame with crime type counts and occurrences
crime_sctype_df = pd.DataFrame({'Count': crime_counts_by_type, 'Age': crime_counts_by_age})

plt.figure(figsize=(8, 6))
sns.regplot(data=crime_sctype_df, x='Age', y='Count', scatter=true)

plt.xlabel('Victim Age')
plt.ylabel('Count of Specific Crime Type')
plt.title('Correlation between Victim Age and Count of Specific Crime Type')
plt.show()

NameError: name 'true' is not defined

<Figure size 800x600 with 0 Axes>

In [ ]:
# Visualization 13: 
# Histogram or kernel density plot showing the distribution of victim ages.

crime_data['Vict Age'].plot(kind='hist', bins=20)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Distribution of Victim Ages')
plt.show()

In [ ]:
# Visualization 14: 
# Stacked bar plot or heat map showing the correlation between crime types and victim demographics.

crime_data_cross = pd.crosstab(crime_data['Victim Demographics'], crime_data['Crime Type'])
crime_data_cross.plot(kind='bar', stacked=True)
plt.xlabel('Victim Demographics')
plt.ylabel('Count')
plt.title('Crime Type Distribution by Victim Demographics')
plt.legend(title='Crime Type')
plt.show()